In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Basic SAW generator

In [ ]:
# Returns a list of possible next coordinates
def get_candidate_moves(coord, visited):
    candidates = []
    
    for dx, dy in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
        new_coord = (coord[0] + dx, coord[1] + dy)
        if new_coord not in visited:
            candidates.append((dx, dy))
    return candidates


# Returns the coordinates visited in order of when they were visited
# and the moves made to get there, as well as whether the SAW is valid
def generate_loose_saw(L):
    coords = [(0, 0)]
    moves = []
    visited = set(coords)
    while len(coords) <= L:
        candidates = get_candidate_moves(coords[-1], visited)
        if len(candidates) == 0: # no possible moves, keep still
            coords += [coords[-1]]*(L+1-len(coords))
            break
        move = random.choice(candidates)
        coords.append((coords[-1][0] + move[0], coords[-1][1] + move[1]))
        moves.append(move)
        visited.add(coords[-1])
    return coords, moves, len(moves) == L


def generate_valid_saw(L):
    while True:
        coords, moves, valid = generate_loose_saw(L)
        if valid:
            return coords, moves


In [ ]:
generate_loose_saw(10)

([(0, 0),
  (-1, 0),
  (-1, -1),
  (-1, -2),
  (-2, -2),
  (-3, -2),
  (-3, -1),
  (-3, 0),
  (-2, 0),
  (-2, -1),
  (-2, -1)],
 [(-1, 0),
  (0, -1),
  (0, -1),
  (-1, 0),
  (-1, 0),
  (0, 1),
  (0, 1),
  (1, 0),
  (0, -1)],
 False)

# Recursive SAW formulation

Let $B(L_A, L_B)$ be the probability that when sampling SAWs of length $L_A$ and $L_B$ independently and uniformly at random, their concatenation is still a SAW.

$B(L_A, L_B) = {c_{L_A + L_B} \over c_{L_A} c_{L_B}}$

$\mu^{L_A} \approx {c_{L_A + L_B} \over c_{L_B}} = B(L_A, L_B) c_{L_A}$

We can generate samples of SAWs of length $L_A$ and $L_B$ to estimate $B(L_A, L_B)$

In [318]:
def estimate_B(Z_A, Z_B):
    if len(Z_A) != len(Z_B):
        raise ValueError("Z_A and Z_B must have the same length")

    Z = zip(Z_A, Z_B)

    failures = 0
    for z_A, z_B in Z:
        visited = set(z_A[0])
        back_moves = z_B[1]
        curr_coord = z_A[0][-1]
        for move in back_moves:
            next_coord = (curr_coord[0] + move[0], curr_coord[1] + move[1])
            if next_coord in visited:
                failures += 1
                break
            else:
                curr_coord = next_coord
                visited.add(next_coord)

    return 1 - (failures / len(Z_A))


# estimates c_{L_A + L_B}
def estimate_c(Z_A, Z_B, c_L_A, c_L_B):
    return estimate_B(Z_A, Z_B) * (c_L_A * c_L_B)


def estimate_mu(C):
    mus = []
    for c1, c2 in zip(C[:-1], C[1:]):
        mus.append((c2[1]/c1[1]) ** (1 / c1[0]))
    return mus


In [310]:
estimate_c([generate_valid_saw(10) for _ in range(10_000)],\
           [generate_valid_saw(10) for _ in range(10_000)],\
            44100, 44100)

852604704.0

In [325]:
C = [(10, 44100)]
n = 10_000
stopping_condition = 640 # stops when the last L is less than this value

while C[-1][0] < stopping_condition:
    L, c_L = C[-1]
    print(f'Working on L = {2*L}...')
    Z_A = [generate_valid_saw(L) for _ in range(n)]
    Z_B = [generate_valid_saw(L) for _ in range(n)]
    C.append((L*2, estimate_c(Z_A, Z_B, c_L, c_L)))

C

Working on L = 20...
Working on L = 40...
Working on L = 80...
Working on L = 160...
Working on L = 320...
Working on L = 640...


[(10, 44100),
 (20, 828294578.9999999),
 (40, 2.2606069421345936e+17),
 (80, 1.2244383617397778e+34),
 (160, 2.7211374825854835e+67),
 (320, 9.833294456446849e+133),
 (640, 9.824077874506336e+266)]

In [326]:
estimate_mu(C)

[2.6753131420310123,
 2.6412043987917544,
 2.6202504049025084,
 2.6111701323564667,
 2.606076610705741,
 2.6038965569717685]

# Pivot

In [ ]:
# Rotates the moves by 90, 180, or 270 degrees
# angle = 90, 180, or 270 degrees
def rotate(moves, angle):
    match angle:
        case 90:
            for move in moves:
                yield (move[1], -move[0])
        case 180:
            for move in moves:
                yield (-move[0], -move[1])
        case 270:
            for move in moves:
                yield (-move[1], move[0])
        case _:
            raise ValueError("Angle must be 90, 180, or 270 degrees.")


# Reflects the moves across the x or y axis
# dim = 0 for x-axis, dim = 1 for y-axis
def reflect(moves, dim):
    if dim == 0:
        for move in moves:
            yield (move[0], -move[1])
    elif dim == 1:
        for move in moves:
            yield (-move[0], move[1])
    else:
        raise ValueError("Axis must be 'x' or 'y'.")


In [328]:
def generate_saw_pivot(saw, generator):
    coords, moves = saw
    successes = 0
    while successes < len(moves):
        pivot = random.randint(0, len(moves)-1)
        proposed_moves = saw[1][:pivot] + generator(saw[1][pivot:])
        proposed_coords = coords[:pivot+1]
        visited = set(proposed_coords)
        for move in proposed_moves:
            next_coord = (proposed_coords[-1][0] + move[0], proposed_coords[-1][1] + move[1])
            if next_coord in visited:
                break
            proposed_coords.append(next_coord)
            visited.add(next_coord)
        if len(visited) == len(moves)+1:
            coords, moves = proposed_coords, proposed_moves
            successes += 1
    return coords, moves


def generate_saw_pivot_batch(saw, generator, n):
    saws = [saw]
    while len(saws) < n:
        saws.append(generate_saw_pivot(saws[-1], generator))
    return saws


In [ ]:
generate_saw_pivot_batch(generate_valid_saw(10), lambda moves: rotate(moves, 90), 2)

[([(0, 0),
   (-1, 0),
   (-1, 1),
   (-2, 1),
   (-3, 1),
   (-4, 1),
   (-5, 1),
   (-5, 0),
   (-4, 0),
   (-3, 0),
   (-3, -1)],
  [(-1, 0),
   (0, 1),
   (-1, 0),
   (-1, 0),
   (-1, 0),
   (-1, 0),
   (0, -1),
   (1, 0),
   (1, 0),
   (0, -1)]),
 ([(0, 0),
   (0, 1),
   (1, 1),
   (1, 2),
   (1, 3),
   (1, 4),
   (1, 5),
   (0, 5),
   (0, 4),
   (0, 3),
   (-1, 3)],
  [(0, 1),
   (1, 0),
   (0, 1),
   (0, 1),
   (0, 1),
   (0, 1),
   (-1, 0),
   (0, -1),
   (0, -1),
   (-1, 0)])]

In [ ]:
def custom_generator(moves):
    chosen_generator = random.choice([
        rotate(moves, 90),
        rotate(moves, 180),
        rotate(moves, 270),
        reflect(moves, 0),
        reflect(moves, 1),
    ])
    yield from chosen_generator(moves)

In [344]:
generate_saw_pivot_batch(generate_valid_saw(10), custom_generator, 10000)

[([(0, 0),
   (0, -1),
   (0, -2),
   (-1, -2),
   (-1, -3),
   (-1, -4),
   (-2, -4),
   (-3, -4),
   (-4, -4),
   (-4, -3),
   (-3, -3)],
  [(0, -1),
   (0, -1),
   (-1, 0),
   (0, -1),
   (0, -1),
   (-1, 0),
   (-1, 0),
   (-1, 0),
   (0, 1),
   (1, 0)]),
 ([(0, 0),
   (-1, 0),
   (-2, 0),
   (-2, 1),
   (-3, 1),
   (-4, 1),
   (-4, 2),
   (-4, 3),
   (-4, 4),
   (-3, 4),
   (-3, 3)],
  [(-1, 0),
   (-1, 0),
   (0, 1),
   (-1, 0),
   (-1, 0),
   (0, 1),
   (0, 1),
   (0, 1),
   (1, 0),
   (0, -1)]),
 ([(0, 0),
   (0, 1),
   (0, 2),
   (1, 2),
   (1, 3),
   (1, 4),
   (2, 4),
   (3, 4),
   (4, 4),
   (4, 3),
   (3, 3)],
  [(0, 1),
   (0, 1),
   (1, 0),
   (0, 1),
   (0, 1),
   (1, 0),
   (1, 0),
   (1, 0),
   (0, -1),
   (-1, 0)]),
 ([(0, 0),
   (0, -1),
   (0, -2),
   (-1, -2),
   (-1, -3),
   (-1, -4),
   (-2, -4),
   (-3, -4),
   (-4, -4),
   (-4, -3),
   (-3, -3)],
  [(0, -1),
   (0, -1),
   (-1, 0),
   (0, -1),
   (0, -1),
   (-1, 0),
   (-1, 0),
   (-1, 0),
   (0, 1),
   (1, 0

In [346]:
C_pivot = [(10, 44100)]
n = 10_000
stopping_condition = 640 # stops when the last L is less than this value

while C_pivot[-1][0] < stopping_condition:
    L, c_L = C_pivot[-1]
    print(f'Working on L = {2*L}...')
    Z_A = generate_saw_pivot_batch(generate_valid_saw(L), custom_generator, n)
    Z_B = generate_saw_pivot_batch(generate_valid_saw(L), custom_generator, n)
    C_pivot.append((L*2, estimate_c(Z_A, Z_B, c_L, c_L)))

C_pivot

Working on L = 20...
Working on L = 40...
Working on L = 80...


KeyboardInterrupt: 